<a href="https://colab.research.google.com/github/Mellow-py/Data-Analysis-Project/blob/main/DSNHACK21.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install lightgbm==3.2.1

     |████████████████████████████████| 2.0 MB 5.3 MB/s 
  Attempting uninstall: lightgbm
    Found existing installation: lightgbm 2.2.3
    Uninstalling lightgbm-2.2.3:
      Successfully uninstalled lightgbm-2.2.3


In [2]:
import numpy as np
import pandas as pd

from lightgbm import LGBMRegressor, LGBMClassifier
from xgboost import XGBRegressor, XGBClassifier
# from catboost import CatBoostRegressor, CatBoostClassifier
from sklearn.linear_model import Ridge ,LinearRegression
import imblearn
from tqdm import tqdm_notebook
from sklearn.model_selection import KFold,StratifiedKFold ,GroupKFold, cross_val_score
from sklearn.metrics import mean_squared_error, f1_score, confusion_matrix, classification_report, recall_score, precision_recall_fscore_support, precision_score
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans

pd.set_option('max_column', 10000)
pd.set_option('max_row', 10000)

# Plot Packages
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn-darkgrid')

import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [5]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [7]:
from google.colab import files
uploaded = files.upload()

Saving SampleSubmission.csv to SampleSubmission.csv
Saving Test.csv to Test.csv
Saving Train.csv to Train.csv


In [8]:
# Load files
# path = '/My Drive/AI/' #This is the path for of the folder where the datasets is stored in Drive
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')
sub = pd.read_csv('SampleSubmission.csv')

In [9]:
a = train[train['Year_of_Birth']<1940].index
train.drop(a, axis=0, inplace=True)
train.reset_index(drop=True, inplace=True)

In [10]:
len_train = len(train)
df = pd.concat([train, test]).reset_index(drop=True)

In [11]:
df['Disposable_Income'] = df['Disposable_Income'].fillna(train['Disposable_Income'].median())

In [12]:
df['Date_Customer'] = pd.to_datetime(df['Date_Customer'])

df['enrollment_year'] = df['Date_Customer'].dt.year
df['enrollment_month'] = df['Date_Customer'].dt.month
df['enrollment_day'] = df['Date_Customer'].dt.day

In [13]:
df.head()

,ID,Year_of_Birth,Education_Level,Marital_Status,Disposable_Income,No_of_Kids_in_home,No_of_Teen_in_home,Date_Customer,Recency,Discounted_Purchases,WebPurchases,CatalogPurchases,StorePurchases,Amount_on_Wines,Amount_on_Fruits,Amount_on_MeatProducts,Amount_on_FishProducts,Amount_on_SweetProducts,Amount_on_GoldProds,WebVisitsMonth,Cmp3Accepted,Cmp4Accepted,Cmp5Accepted,Cmp1Accepted,Cmp2Accepted,Any_Complain,Response,enrollment_year,enrollment_month,enrollment_day
0,ID_4A9AR6FQ,1955,Graduation,Divorced,77504.4,1,1,2014-06-22,56,2,1,1,4,71,3,16,0,6,20,3,0,0,0,0,0,0,0.0,2014,6,22
1,ID_X28T3VEK,1958,Graduation,Together,56784.0,0,1,2013-01-08,17,6,5,1,8,205,22,111,50,15,19,6,0,0,0,0,0,0,0.0,2013,1,8
2,ID_AWXARH57,1962,Graduation,Single,103714.8,0,0,2013-11-21,17,0,7,4,7,469,31,873,29,18,37,2,0,0,0,1,0,0,1.0,2013,11,21
3,ID_FQVZHE81,1979,2n Cycle,Single,46311.6,1,0,2013-09-13,49,3,4,1,3,56,15,49,17,24,24,8,0,0,0,0,0,0,0.0,2013,9,13
4,ID_QVLWGPRN,1959,Graduation,Married,87486.0,0,0,2014-01-21,59,1,3,7,9,412,117,445,181,165,203,1,0,0,0,1,0,0,0.0,2014,1,21


In [14]:
cat_col = ['Education_Level','Marital_Status']

for i in cat_col:
    le = LabelEncoder()
    df[i] = le.fit_transform(df[i])

In [15]:
train = df[:len_train]
test = df[len_train:]

In [16]:
X = train.drop(columns=['ID','Date_Customer','Response'])
y = train['Response']
tes = test.drop(columns=['ID','Date_Customer','Response'])

In [17]:
y.value_counts()

0.0    1328
1.0     239
Name: Response, dtype: int64

In [18]:
from imblearn.over_sampling import SMOTE
seed = 50
sm = SMOTE(sampling_strategy={1:280, 0:1328}, random_state=seed) 
X_, y_ = sm.fit_resample(X, y)

In [20]:
%%time
from sklearn.ensemble import RandomForestClassifier
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=1234)
oofs = np.zeros((len(X_)))
preds = np.zeros((len(tes)))

for fold_, (trn_idx, val_idx) in enumerate(folds.split(X_, y_)):
    
    X_trn, y_trn = X_[trn_idx], y_[trn_idx]
    X_val, y_val = X_[val_idx], y_[val_idx]
    
    
    clf = LGBMClassifier(learning_rate=0.2, n_estimators=1000, scale_pos_weight=8, max_depth=-1, colsample_bytree=0.7, subsample=0.85, reg_alpha=3, reg_lambda=20)
    clf.fit(X_trn, y_trn, eval_set = [(X_val, y_val)],
                    verbose = 0, early_stopping_rounds=200)
    
    vp = clf.predict_proba(X_val)[:,1]
    b = pd.DataFrame(vp)
    b[0] = np.where(b[0]>0.5, 1, 0)
    val_score = f1_score((y_val), (b[0]))
    
    print(f'\nFold {fold_+1} Val score: {val_score}\n')
    
    tp = clf.predict_proba(tes)[:,1]
    
    oofs[val_idx] = vp
    preds += tp/5
    
a = pd.DataFrame(oofs)
a[0] = np.where(a[0]>0.5, 1, 0)
oof_score = f1_score((y_), (a[0]))
print(f'\nOOF F1 score is : {oof_score}')


Fold 1 Val score: 0.5486725663716814


Fold 2 Val score: 0.6666666666666666


Fold 3 Val score: 0.6507936507936508


Fold 4 Val score: 0.734375


Fold 5 Val score: 0.7359999999999999


OOF F1 score is : 0.66996699669967
CPU times: user 2.03 s, sys: 88.7 ms, total: 2.12 s
Wall time: 1.11 s


In [21]:
sub['Response'] = preds

In [22]:
sub.head()

,ID,Response
0,ID_ZPMABNVX,0.064083
1,ID_WFE91NAA,0.578328
2,ID_JV11RBRK,0.021324
3,ID_6B7SVKY9,0.670002
4,ID_GOVUZ545,0.241454


In [23]:
sub['Response'] = np.where(sub['Response']>=0.5, 1, 0)

In [24]:
sub['Response'].value_counts()

0    561
1    111
Name: Response, dtype: int64

In [26]:
sub.to_csv('/drive/My Drive/AI/.csv', index=False)

FileNotFoundError: ignored